## IMPORTS

In [11]:
import numpy as np
import pandas as pd
import datetime
import os


# APIs and data requests
import alpha_vantage
import quandl
import requests
import json



## CLASSES

In [12]:
class Interface(object):
    
    def __init__(self, ann, dataDict):
        '''
        parameters:
            ann - neural network model
            dataDict - dict of data objects containing all prediction relevant data for every chosen stock 
        '''
        self.ann = ann
        self.data = dataDict
        
    
    def run(self, stock):
        '''
        test-run the trained ANN on live data - visualize and print results.
        parameters:
            stock - string of the stocks ticker symbol
        '''
        dataVec = self.data[stock].getDataCurrent() # get data vector for current timestamp 
        pred = self.ann.predict(dataVec) # ann price prediction for next timestamp
        self.data.updateData()  # update data to next (predicted) timestamp
        obs = self.data[stock].getPrice()  # get price from new timestamp
        self.ann.learn(dataVec, pred, obs)  # online learning for ANN
        diff = np.abs(pred - obs)  # calc difference between prediction and observation
        
        # TODO: some visualization and evaluation of accuracy over time
        
        
    def train(self):
        '''
        pre-train the neural network on historical data.       
        '''
        self.ann.train(self.data) # call train method on ANN object
 

In [13]:
class ANN(object):
    
    def __init__(self, modelPath=None, weightPath=None):
        #self.model = self.buildModel(modelPath, weightPath)
        pass
        
    def buildModel(self, modelPath, weightPath):
        '''
        read safed model and weights or create new one if none exists.
        parameters: 
            modelPath - path of the ann model
            weightPath - path of the ann weights
        '''
        if os.path.exists(modelPath):
            # read model
            if os.path.exists(weightPath):
                # read weights
                pass
        else:
            # build new model
            pass
            
        # compile and return model
        pass        
        
    def predict(self, dataVec):
        '''
        use models predict method to predict price from data.
        parameters:
            dataVec - vector of current data
        returns:
            price - predicted price
        '''
        # return self.model.predict(dataVec)
        pass
    
    def train(self, data):
        '''
        train model on historical data.
        parameters: 
            data - dict of data of all stocks chosen 
        '''
        # going through all stocks 
        # get predData for the stock - vector of all prices
        # self.model.fit(data[stock], predData)
        pass
        
    def learn(self, data, predPrice, obsPrice):
        '''
        online learning.
        parameters: 
            data - current data used for prediction
            predPrice - price predicted
            obsPrice - price observed
        '''
        # self.model.fit(data, obs)
        pass
    
    def test(self, data, start, end):
        '''
        check accuracy of the model on random timespan.
        parameters: 
            data - historical data
            start - start timestamp of random timespan
            end - end timestamp of random timespan
        returns:
            accuracy - accuracy measure of predictions
        '''
        # predictions are price of (data + 1 timestep)
        
        # return accuracy
        pass 
    
    

In [14]:
class economy(object):
    '''
    object holding all relevant economic data as numerically encoded values (historical, current).
    https://blog.quandl.com/api-for-economic-data
    Federal Reserve Economic Data (FRED)
    key: cEnL78yMi4hkJTQ1cLy_
    '''
    
    def __init__(self):
        self.update()
    
    def update(self):
        self.frame = pd.read_csv('./datafiles/economic_data.csv', index_col=0)

In [15]:
class finances(object):
    '''
    object holding all relevant financial data as numerically encoded values (historical, current)
    '''
    
    def __init__(self, stock):
        self.stock = stock
        self.update()
    
    def update(self):
        self.frame = pd.read_csv('./datafiles/{0}/{0}.csv'.format(self.stock), index_col=0)



In [21]:
class price(object):
    '''
    object holding all relevant price data as numerically encoded values (historical, current).
    https://www.alphavantage.co/documentation/ API used for price data (key: E8X1JONC18APVAQW).
    '''

    def __init__(self, stock, start_date_data):
        self.stock = stock
        self.update(start_date_data)
        
    
    def update(self, start_date_data):
        API_URL = "https://www.alphavantage.co/query"

        data1 = {
            "function": "TIME_SERIES_DAILY",
            "symbol": "AAPL",
            "outputsize": "full",
            "datatype": "json",
            "apikey": "E8X1JONC18APVAQW"
            }

        response = requests.get(API_URL, data1)
        data = response.json()
        metaData = data.pop("Meta Data", None)
        df = pd.DataFrame.from_dict(data, orient="columns")
        df = df["Time Series (Daily)"].apply(pd.Series).apply(pd.to_numeric)
        self.price = df["4. close"].to_frame('price') # save closing prices as series over dates as index
        self.price = self.price.loc[start_date_data:]
        
        
        data2 = {
            "function": "TIME_SERIES_DAILY",
            "symbol": "^GSPC",
            "outputsize": "full",
            "datatype": "json",
            "apikey": "E8X1JONC18APVAQW"
            }
        
        response = requests.get(API_URL, data2)
        data = response.json()
        metaData = data.pop("Meta Data", None)
        df = pd.DataFrame.from_dict(data, orient="columns")
        df = df["Time Series (Daily)"].apply(pd.Series).apply(pd.to_numeric)
        self.s_p_price = df["4. close"].to_frame('S&P500 price') # save closing prices as series over dates as index
        self.s_p_price = self.s_p_price.loc[start_date_data:]
        
        self.frame = pd.concat([self.price,self.s_p_price], axis=1, sort=False)

In [22]:
class sentiment(object):
    '''object holding all relevant sentiment data as numerically encoded values (historical, current)'''

    def __init__(self, stock, start_date_data):
        pass
    
    def update():
        pass

In [23]:
class Data(object):
    '''data object holding all relevant data for prediction'''
    def __init__(self, stock, start_date_data):
        self.stock = stock
        self.economic = economy()
        self.financial = finances(stock)
        self.price = price(stock, start_date_data)
        self.sentiment = sentiment(stock, start_date_data)
        self.frame = pd.concat([self.economic.frame, self.financial.frame, self.price.frame], axis=1, sort=False)
        
    def getDataCurrent(self):
        '''loop over all subobject and their attributes and save the values in a vector.'''
        pass
    
    def getDataHistory(self):
        '''loop over all subobject and their attributes and save the values for every timestep in a matrix.'''
        pass
    
    def updateData(self, start_date_data):
        '''update data.'''
        self.priceData.update(start_date_data)
        self.sentiment.update(start_date_data)
    
    def getPrice(self, timestamp):
        '''
        returns price of stock in dollar.
        parameters:
            timestamp - timestamp for which to get the price (default is current timestamp)
        '''
        self.priceData.price(timestamp)
        return price 

### Run code (MAIN METHOD)

In [24]:
stocks = ['MSFT']
start_date = "2015-12-01"
ann = ANN()
data = {}
for stock in stocks:
    data[stock] = Data(stock, start_date) 
interface = Interface(ann, data)


In [25]:
interface.data['MSFT'].frame

,gdp,interest_rates,unemployment,inflation,quarterly indicator,number of months last report period,operating revenue,total revenue,adjustments to revenue,cost of sales,...,(increase) decrease in other current assets,net cash from financing activities,income per employee,sales per employee,purchase of long-term investments,"other financing charges, net",net cash from investing activities,repayment of long-term debt,price,S&P500 price
2015-12-01,NaN,0.25,7907.0,NaN,4.0,3.0,23796.0,23796.0,0.0,9872.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.34,2102.6299
2015-12-02,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.28,2079.5100
2015-12-03,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.20,2049.6201
2015-12-04,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.03,2091.6899
2015-12-05,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-06,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-07,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.28,2077.0701
2015-12-08,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.23,2063.5901
2015-12-09,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.62,2047.6200
2015-12-10,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.17,2052.2300


#### ideas

- cashfromoperatingactivities
- netincome
- researchanddevelopmentexpense
- totalassets
- totalliabilities
- book value = totalassets - intagibleassets - totalliabilities
- P/E = price / netincome 
- P/B = price / book value 
